# Scoring Elaticsearch Results
## LMI Capstone Team
## Summer Chambers | Steve Morris | Kaleb Shikur

### Setup

In [50]:
import pandas as pd
import numpy as np
import json
import math
from statistics import mode
from sklearn.metrics import ndcg_score, f1_score, average_precision_score
from matplotlib import pyplot as plt
import seaborn as sns

In [51]:
metadata_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/CMS-1701-P%20Comment%20MetaData.csv"
data = pd.read_csv(metadata_url, usecols=range(0,36))[:469] #ignore last few columns and blank rows at end of csv 
data = data.rename(columns=lambda x: x.strip()) #strip whitespace from columns

In [67]:
data10 = data.fillna(0) #fill NaN with 0

section_cols = data10.columns[3:] 
data10[section_cols] = data10[section_cols].replace(["Y"], 1) #replace Y with 1 in approriate columns
data11 = data10.copy()
section_cols1  = data11.columns[3:] 
data11[section_cols1] = np.where((data11[section_cols1]  != 1), 0, data11[section_cols1])

# Combining columns for index matching: (A6b, A6b.1, = A6b),  (C3b, C3b.1'= C3b) ('A7', 'A7.1', 'A7.2', = A7b, a7c),  (F = F2, F3)

data11['A6b'] = (data11['A6b'] + data11['A6b.1'])
data11['A6b'] = data11['A6b'].replace(2,1)
data11['C3b'] = (data11['C3b'] + data11['C3b.1'])
data11['C3b'] = data11['C3b'].replace(2,1)
data11['A7'] = (data11['A7'] + data11['A7.1'] + data11['A7.2'])
data11['A7'] = data11['A7'].replace(2,1)
data11['A7'] = data11['A7'].replace(3,1)

data11 = data11.drop(['A6b.1', 'C3b.1', 'A7.1', 'A7.2'], axis=1)
data11 = data11[0:468]
data11.Name = [name.split('DRAFT-')[1].split('-')[0] for name in data11.Name]
data11 = data11.rename(columns=lambda x: x.lower())
section_cols1 = data11.columns[3:]

In [68]:
data11

,name,organization name / submitter name,submitter state,a2,a3,a4b,a4c,a5b,a5c,a5d,...,d3c,d3d,d4,e2,e3,e4,e5,e6,e7,f
0,0002,Erick Meleher,NC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0003,Mayank Shah,IL,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,0004,Mayank Shah,IL,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0005,Morey Menacker,NJ,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0006,Todd Rapoza,MA,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,0465,Morehouse Choice Accountable Care Organization...,GA,1,1,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
464,0466,Liberty ACO,TX,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
465,0467,Washington State Hospital Association,WA,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
466,0468,Think Whole Person Healthcare,NE,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
truth_dict = {} # initiate dict
for i, name in enumerate(data11.name): # iterate through Comment Names
    dict1 = {col:data11[col].iloc[i] for col in section_cols1} # locate this row's values for each column
    list1 = [key for key,value in dict1.items() if value==1] # create list of matching sections for this row
    truth_dict[name] = list1 # add Comment Name and list of matching sections to dict

In [16]:
with open("unique_comments2018.json") as f:
    unique_comments2018 = json.load(f)

In [63]:
unique_data = data11.loc[data11['name'].isin(unique_comments2018.keys())]

In [64]:
unique_truth_dict = {} # initiate dict
for i, name in enumerate(unique_data.name): # iterate through Comment Names
    dict1 = {col:unique_data[col].iloc[i] for col in section_cols1} # locate this row's values for each column
    list1 = [key for key,value in dict1.items() if value==1] # create list of matching sections for this row
    unique_truth_dict[name] = list1 # add Comment Name and list of matching sections to dict

In [65]:
dump = json.dumps(unique_truth_dict)
f = open('unique_truth_dict.json','w')
f.write(dump)
f.close()

In [66]:
num_matches = {key: len(value) for key, value in unique_truth_dict.items()}
mode(list(num_matches.values()))

1

In [69]:
#define true rule section names
truth_keys = list(section_cols1)
truth_keys = [key1.lower() for key1 in truth_keys]

## Header Index Testing

In [ ]:
with open('header_results.json') as f:
    header_results_json = json.load(f)

In [ ]:
#tranlate ES keys to true rule section names, MAINTAINING ORDER
for key, value in header_results_json.items():
    value_list = []
    for val_idx in value:
        for true_key in truth_keys:
            if true_key in val_idx and true_key not in value_list: 
                value_list.append(true_key)
        header_results_json[key]=value_list     

In [ ]:
# match_dict shows all true positives, without penalty to false categorization
match_dict= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), header_results_json.items()):
    match_count = 0
    for val_idx in value1:
        if val_idx in value2:
            match_count += 1
        match_dict[key1]=match_count/len(value1)

In [ ]:
# unpenalized accuracy of true positives
meanvals = np.mean(list(match_dict.values()))
meanvals

In [ ]:
# match_dict1 shows all true positives, with penalty to false categorization
match_dict1= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), header_results_json.items()):
    match_count = 0
    for val_idx in value2:
        if val_idx in value1:
            match_count += 1
        match_dict1[key1]=match_count/len(value2)

In [ ]:
# penalized accuracy of true positives
meanvals1 = np.mean(list(match_dict1.values()))
meanvals1

In [ ]:
np.mean([meanvals,meanvals1])

### Sklearn NDCG Headers

In [ ]:
#create truth dict with values equal to sparse array length=29, binary
truth_dict_all = truth_dict.copy()
for key, value in truth_dict.items():
    new_value_list = np.zeros(len(truth_keys))
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                new_value_list[i] = 1
    truth_dict_all[key] = new_value_list

In [ ]:
#create header results dict with values equal to sparse array length=29, binary
binary_header_results_all = header_results_json.copy()
for key, value in header_results_json.items():
    new_value_list = np.zeros(len(truth_keys))
    counter = 1
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                counter += 1
                new_value_list[i] = 1
    binary_header_results_all[key] = new_value_list

In [ ]:
#create header results dict with values equal to sparse array length=29, (discounted)
header_results_all = header_results_json.copy()
for key, value in header_results_json.items():
    new_value_list = np.zeros(len(truth_keys))
    counter = 1
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                counter += 1
                new_value_list[i] = 1/math.log(counter, 2)
    header_results_all[key] = new_value_list

In [ ]:
headers_ndcg = {}
headers_ap = {}
headers_f1 = {}
for (true_key, true_value), (es_key, es_value) in zip(truth_dict_all.items(), header_results_all.items()):
    headers_ndcg[true_key] = ndcg_score([true_value], [es_value])
    headers_ap[true_key] = average_precision_score(true_value, es_value, pos_label=0)
for (true_key, true_value), (es_key, es_value) in zip(truth_dict_all.items(), binary_header_results_all.items()):
    headers_f1[true_key] = f1_score(true_value, es_value, zero_division=0, pos_label=0)

In [ ]:
headers_eval = pd.DataFrame(data=[headers_ndcg, headers_ap, headers_f1]).T
headers_eval = headers_eval.rename({0:"NDCG", 1:"AP", 2:"F1"}, axis=1)

In [ ]:
plt.figure(figsize = (14,10))
sns.kdeplot(headers_eval.NDCG, alpha=.1, shade=True, label='NDCG')
sns.kdeplot(headers_eval.AP, alpha=.1, shade=True,label='AP')
sns.kdeplot(headers_eval.F1,  alpha=.1, shade=True,label='F1')
plt.legend()
plt.title('Ranking Metric Performance, Headers Index', fontsize=20)
plt.xlabel('Score Distribution', fontsize=16)
plt.ylabel('Score Density', fontsize=16)
plt.xlim(0,1);

### Which comments were scored poorly?

In [ ]:
with open("comments2018.json") as f:
    comments2018 = json.load(f)

In [ ]:
poorf1 = headers_eval.query("F1 < 0.5").F1
poorf1

In [ ]:
poorndcg = headers_eval.query("NDCG < 0.3 & NDCG > 0").NDCG
poorndcg

In [ ]:
poorf1_comments = {key:comment for key, comment in comments2018.items() if key in poorf1}
{key:len(comment) for key, comment in poorf1_comments.items()}

In [ ]:
poorf1_comments_truth = {key:matches for key, matches in truth_dict.items() if key in poorf1}
poorf1_comments_truth

In [ ]:
poorndcg_comments = {key:comment for key, comment in comments2018.items() if key in poorndcg}
{key:len(comment) for key, comment in poorndcg_comments.items()}

In [ ]:
poorndcg_comments_truth = {key:matches for key, matches in truth_dict.items() if key in poorndcg}
poorndcg_comments_truth

## Headers Unique Index Testing

In [34]:
with open('unique_header_results.json') as f:
    unique_header_results = json.load(f)

In [35]:
#tranlate ES keys to true rule section names, MAINTAINING ORDER
for key, value in unique_header_results.items():
    value_list = []
    for val_idx in value:
        for true_key in truth_keys:
            if true_key in val_idx and true_key not in value_list: 
                value_list.append(true_key)
        unique_header_results[key]=value_list     

In [36]:
unique_header_results

{'0002': ['a4c', 'a6c', 'a5c', 'c3b', 'c2', 'a3', 'a5b', 'a2', 'e4', 'a7'],
 '0003': ['a4c', 'a6c', 'a3', 'a5c', 'e4', 'c3b', 'a5b', 'a2', 'c2', 'a7'],
 '0004': ['a4c', 'a6c', 'a5c', 'a3', 'e4', 'a7', 'c2', 'a2', 'a5b', 'c3b'],
 '0005': ['a4c', 'a6c', 'a5b', 'a5c', 'c2', 'c3b', 'a3', 'e4', 'a2', 'b2a'],
 '0006': ['a4c', 'a6c', 'a5c', 'a3', 'a2', 'a7', 'a5b', 'c2', 'b2a'],
 '0007': ['a4c', 'a6c', 'c3b', 'a5c', 'a5b', 'c2', 'e4', 'a3', 'a7', 'a2'],
 '0008': ['a4c', 'a6c', 'c3b', 'a5b', 'c2', 'a5c', 'a3', 'e4', 'b2a', 'b2b'],
 '0009': ['a4c', 'a6c', 'c3b', 'a5c', 'a5b', 'a3', 'e4', 'a7', 'c2', 'b2a'],
 '0010': ['a4c', 'a6c', 'a5b', 'a5c', 'a3', 'c3b', 'e6', 'a2', 'b2a', 'e4'],
 '0011': ['a4c', 'a6c', 'a5c', 'a5b', 'a3', 'c2', 'c3b', 'e4', 'a7', 'a2'],
 '0012': ['a4c', 'a6c', 'a5c', 'a5b', 'a3', 'c2', 'c3b', 'e4', 'a7', 'a2'],
 '0013': ['a4c', 'a6c', 'a5c', 'a5b', 'a3', 'c2', 'c3b', 'e4', 'a7', 'a2'],
 '0014': ['a4c', 'a6c', 'a5c', 'a5b', 'a3', 'c2', 'c3b', 'e4', 'a7', 'a2'],
 '0016': ['a4

### Sklearn

In [43]:
#create truth dict with values equal to sparse array length=29, binary
unique_truth_dict_all = unique_truth_dict.copy()
for key, value in unique_truth_dict.items():
    new_value_list = np.zeros(len(truth_keys))
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                new_value_list[i] = 1
    unique_truth_dict_all[key] = new_value_list

In [44]:
#create header results dict with values equal to sparse array length=29, (discounted)
unique_header_results_all = unique_header_results.copy()
for key, value in unique_header_results.items():
    new_value_list = np.zeros(len(truth_keys))
    counter = 1
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                counter += 1
                new_value_list[i] = 1/math.log(counter, 2)
    unique_header_results_all[key] = new_value_list

In [45]:
#create header results dict with values equal to sparse array length=29, binary
unique_binary_header_results_all = unique_header_results.copy()
for key, value in unique_header_results.items():
    new_value_list = np.zeros(len(truth_keys))
    counter = 1
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                counter += 1
                new_value_list[i] = 1
    unique_binary_header_results_all[key] = new_value_list

In [46]:
unique_headers_ndcg = {}
unique_headers_ap = {}
unique_headers_f1 = {}
for (true_key, true_value), (es_key, es_value) in zip(unique_truth_dict_all.items(), unique_header_results_all.items()):
    unique_headers_ndcg[true_key] = ndcg_score([true_value], [es_value])
    unique_headers_ap[true_key] = average_precision_score(true_value, es_value, pos_label=0)
for (true_key, true_value), (es_key, es_value) in zip(unique_truth_dict_all.items(), unique_binary_header_results_all.items()):
    unique_headers_f1[true_key] = f1_score(true_value, es_value, zero_division=0, pos_label=0)

In [47]:
unique_headers_eval = pd.DataFrame(data=[unique_headers_ndcg, unique_headers_ap, unique_headers_f1]).T
unique_headers_eval = unique_headers_eval.rename({0:"NDCG", 1:"AP", 2:"F1"}, axis=1)

In [49]:
unique_headers_eval.describe()

,NDCG,AP,F1
count,85.000000,85.000000,85.000000
mean,0.517658,0.800936,0.736477
std,0.266236,0.214707,0.105832
min,0.000000,0.068966,0.190476
25%,0.319753,0.733155,0.702703
50%,0.501266,0.888425,0.765957
75%,0.726067,0.956577,0.800000
max,0.993542,1.000000,0.837209


## Hybrid Index Testing

In [ ]:
with open('hybrid_results.json') as f1:
    hybrid_results_json = json.load(f1)

In [ ]:
#tranlate ES keys to true rule section names, MAINTAINING ORDER
for key, value in hybrid_results_json.items():
    value_list = []
    for val_idx in value:
        for true_key in truth_keys:
            if true_key in val_idx[0:4] and true_key not in value_list:
                    value_list.append(true_key)
        hybrid_results_json[key]=value_list            

In [ ]:
# match_dict shows all true positives, without penalty to false categorization
match_dict= {}
for (key1, value1), (key2, value2) in zip(truth_dict.items(), hybrid_results_json.items()):
    match_count = 0
    for val_idx in value1:
            if val_idx in value2:
                match_count += 1
            match_dict[key1]=match_count/len(value1)

In [ ]:
# unpenalized accuracy of true positives
meanvals = np.mean(list(match_dict.values()))
meanvals

In [ ]:
# match_dict1 shows all true positives, with penalty to false categorization
match_dict1= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), hybrid_results_json.items()):
    match_count = 0
    for val_idx in value2:
        if val_idx in value1:
            match_count += 1
        match_dict1[key1]=match_count/len(value2)

In [ ]:
# penalized accuracy of true positives
meanvals1 = np.mean(list(match_dict1.values()))
meanvals1

In [ ]:
np.mean([meanvals,meanvals1])

### SKlearn NDCG Hybrid

In [ ]:
#create header results dict with values equal to sparse array length=29, binary
binary_hybrid_results_all = hybrid_results_json.copy()
for key, value in hybrid_results_json.items():
    new_value_list = np.zeros(len(truth_keys))
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                new_value_list[i] = 1
    binary_hybrid_results_all[key] = new_value_list

In [ ]:
#create header results dict with values equal to sparse array length=29, binary (discounted)
hybrid_results_all = hybrid_results_json.copy()
for key, value in hybrid_results_json.items():
    new_value_list = np.zeros(len(truth_keys))
    counter = 1
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                counter += 1
                new_value_list[i] = 1/math.log(counter, 2)
    hybrid_results_all[key] = new_value_list

In [ ]:
hybrid_ndcg = {}
hybrid_ap = {}
hybrid_f1 = {}
for (true_key, true_value), (es_key, es_value) in zip(truth_dict_all.items(), hybrid_results_all.items()):
    hybrid_ndcg[true_key] = ndcg_score([true_value], [es_value])
    hybrid_ap[true_key] = average_precision_score(true_value, es_value, pos_label=0)
for (true_key, true_value), (es_key, es_value) in zip(truth_dict_all.items(), binary_hybrid_results_all.items()):
    hybrid_f1[true_key] = f1_score(true_value, es_value, zero_division=0, pos_label=0)

In [ ]:
hybrid_eval = pd.DataFrame(data=[hybrid_ndcg, hybrid_ap, hybrid_f1]).T

In [ ]:
hybrid_eval = hybrid_eval.rename({0:"NDCG", 1:"AP", 2:"F1"}, axis=1)

In [ ]:
Hybrid_mean_scores = hybrid_eval.mean()
Hybrid_mean_scores

In [ ]:
plt.figure(figsize = (14,10))
sns.kdeplot(hybrid_eval.NDCG, alpha=.1, shade=True, label='NDCG')
sns.kdeplot(hybrid_eval.AP, alpha=.1, shade=True,label='AP')
sns.kdeplot(hybrid_eval.F1,  alpha=.1, shade=True,label='F1')
plt.legend()
plt.title('Ranking Metric Performance, Hybrid Index', fontsize=20)
plt.xlabel('Score Distribution', fontsize=16)
plt.ylabel('Score Density', fontsize=16)
plt.xlim(0,1);

### Which comments were scored poorly?

In [ ]:
poorf1 = hybrid_eval.query("F1 < 0.5").F1
poorf1

In [ ]:
poorndcg = hybrid_eval.query("NDCG < 0.3 & NDCG > 0").NDCG
poorndcg

In [ ]:
poorf1_comments = {key:comment for key, comment in comments2018.items() if key in poorf1}
{key:len(comment) for key, comment in poorf1_comments.items()}

In [ ]:
poorf1_comments_truth = {key:matches for key, matches in truth_dict.items() if key in poorf1}
poorf1_comments_truth

In [ ]:
poorndcg_comments = {key:comment for key, comment in comments2018.items() if key in poorndcg}
{key:len(comment) for key, comment in poorndcg_comments.items()}

In [ ]:
poorndcg_comments_truth = {key:matches for key, matches in truth_dict.items() if key in poorndcg}
poorndcg_comments_truth

### NDCG

In [ ]:
def Cummulative_gain(truth, pred):
    counter = 0
    for i in truth:
        if i in pred:
            counter +=1
    return counter
def Discounted_Cummulative_gain(truth, pred):
    DCG = 0
    for i,j in enumerate(pred):
        if j in truth:
            DCG += (1/(math.log(i+2,2)))
    return DCG
def Ideal_discounted_cummulative_gain(truth, pred):
    counter = 0
    IDCG = 0
    for i in truth:
        if i in pred:
            counter +=1
    for i in range(counter):
        IDCG += (1/(math.log(i+2,2)))
    return IDCG
def nDiscounted_Cummulative_gain(truth, pred):
    x = Ideal_discounted_cummulative_gain(truth, pred)
    y = Discounted_Cummulative_gain(truth, pred)
    return y/(x+0.0001)

## Average Precision

In [ ]:
def average_precision(truth, pred):
    runn = 0
    cum_gain = 0
    for i, j in enumerate(truth):
        if j in pred:
            cum_gain +=1
            runn = runn + cum_gain/(i+1)
    average_pred = runn / (cum_gain + 0.0001)
    return average_pred

## F1 Score

In [ ]:
def F_1_score(truth, pred):
    AP = average_precision(truth, pred)
    F_1 = AP/(AP + 1)
    return 2*F_1

## Headers Scoring

In [ ]:
CG = {}
DCG = {}
iDCG = {}
nDCG = {}
AP = {}
F_1 = {}
MAP = 0
count = 0
for (i,(k,j)) in zip(truth_dict.values(),header_results_json.items()):
    CG[k] = Cummulative_gain(i,j)
    DCG[k] = Discounted_Cummulative_gain(i,j)
    iDCG[k] = Ideal_discounted_cummulative_gain(i,j)
    nDCG[k] = nDiscounted_Cummulative_gain(i,j)
    AP[k] = average_precision(i,j)
    F_1[k] = F_1_score(i,j)
    count +=1
    MAP +=AP[k]
Model_eval = pd.DataFrame(data=[CG, DCG, iDCG, nDCG, AP, F_1])
MAP/count

In [ ]:
Model_eval = Model_eval.transpose().rename({0:'CG',1:'DCG',2:'iDCG',3:'nDCG',4:'AP',5:'F1'}, axis=1)
Model_eval

In [ ]:
Mean_scores = Model_eval.mean()
Mean_scores

In [ ]:
plt.figure(figsize = (14,10))

sns.kdeplot(Model_eval.nDCG, alpha=.1, shade=True, label='nDCG')
sns.kdeplot(Model_eval.AP, alpha=.1, shade=True,label='AP')
sns.kdeplot(Model_eval.F1,  alpha=.1, shade=True,label='F1')
plt.legend()
plt.title('Ranking Metric Performance, Headers Index', fontsize=20)
plt.xlabel('Score Distribution', fontsize=16)
plt.ylabel('Score Density', fontsize=16)
plt.xlim(0,1);

## Hybrid Scoring

In [ ]:
CG = {}
DCG = {}
iDCG = {}
nDCG = {}
AP = {}
F_1 = {}
MAP = 0
count = 0
for (i,(k,j)) in zip(truth_dict.values(),hybrid_results_json.items()):
    CG[k] = Cummulative_gain(i,j)
    DCG[k] = Discounted_Cummulative_gain(i,j)
    iDCG[k] = Ideal_discounted_cummulative_gain(i,j)
    nDCG[k] = nDiscounted_Cummulative_gain(i,j)
    AP[k] = average_precision(i,j)
    F_1[k] = F_1_score(i,j)
    count += 1
    MAP += AP[k]
Model_eval2 = pd.DataFrame(data=[CG, DCG, iDCG, nDCG, AP, F_1])
MAP/count

In [ ]:
Model_eval2 = Model_eval2.transpose().rename({0:'CG',1:'DCG',2:'iDCG',3:'nDCG',4:'AP',5:'F1'}, axis=1)
Model_eval2

In [ ]:
Mean_scores2 = Model_eval2.mean()
Mean_scores2

In [ ]:
mean_df = pd.DataFrame(data=[Mean_scores, Mean_scores2])
mean_df=mean_df.rename({0:'headers',1:'hybrid'},axis=0)
mean_df

In [ ]:
plt.figure(figsize = (14,10))

sns.kdeplot(Model_eval2.nDCG, alpha=.1, shade=True, label='nDCG')
sns.kdeplot(Model_eval2.AP, alpha=.1, shade=True, label='AP')
sns.kdeplot(Model_eval2.F1,  alpha=.1, shade=True, label='F1')
plt.legend()
plt.title('Ranking Metric Performance, Hybrid Index', fontsize=20)
plt.xlabel('Score Distribution', fontsize=16)
plt.ylabel('Score Density', fontsize=16)
plt.xlim(0,1);